## Q1. Embedding the query

In [3]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [13]:

model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

query = ["I just discovered the course. Can I join now?"]

embeddings = next(model.embed(query))

embedding_array = np.array(embeddings)

print("Shape:", len(embedding_array))
print("Min value:", embedding_array.min())


Shape: 512
Min value: -0.11726373885183883


## Q2. Cosine similarity with another vector

In [14]:
doc = 'Can I still join the course after the start date?'

embeddings_doc = next(model.embed([doc]))

embedding_doc_array = np.array(embeddings_doc)

cos_sim = np.dot(embedding_array, embedding_doc_array)
print("Cosine similarity:", cos_sim)

Cosine similarity: 0.9008528895674547


## Q3. Ranking by cosine

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [20]:
doc_texts = [doc['text'] for doc in documents]
doc_embeddings = list(model.embed(doc_texts))

similarities = [np.dot(embedding_array, np.array(doc_vec)) for doc_vec in doc_embeddings]

doc_index = int(np.argmax(similarities))

print("Cosine similarities:", similarities)
print("index:", doc_index)

Cosine similarities: [0.7629684696540235, 0.8182378150042888, 0.8085397398734189, 0.7133079015686243, 0.7304499234333599]
index: 1


## Q4. Ranking by cosine, version two

In [22]:
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [23]:
full_embeddings = list(model.embed(full_texts))

full_similarities = [np.dot(embedding_array, np.array(doc_vec)) for doc_vec in full_embeddings]

full_index = int(np.argmax(full_similarities))

print("Cosine similarities:", full_similarities)
print("index:", full_index)

Cosine similarities: [0.8514543236908066, 0.8436594159113069, 0.8408287048502558, 0.7755157969663908, 0.8086007795043937]
index: 0


## Q5. Selecting the embedding model

In [28]:
models_list = TextEmbedding.list_supported_models()

smallest = min(models_list, key=lambda m: m['dim'])
print(smallest['model'], smallest['dim'])


BAAI/bge-small-en 384


## Q6. Indexing with qdrant

In [29]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [30]:
EMBEDDING_DIMENSIONALITY=384

In [31]:
# Define the collection name
collection_name = "ml-zoomcamp"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [35]:
points = []
id = 0
model_handle="BAAI/bge-small-en"
for course in documents_raw:
    for doc in course['documents']:
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model=model_handle),
            payload=doc
        )
        points.append(point)

        id += 1

In [48]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [57]:
results = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text="I just discovered the course. Can I join now?",
        model=model_handle 
    ),
    limit=1, # top closest matches
    with_payload=True #to get metadata in the results
)

score = results.points[0].score
print(score)


0.8703172
